#**Market Reseach App**

An AI-powered application that helps extract insights about a **product**, **company**, or **industry** using web search and summarization.

- We will use duckduckgo-search library, which is lightweight, fast, and doesn’t require an API key.

###**Features:**

**🔍 User Input**

- Accepts a product, brand, or industry name as input.

- Simple and clean input box at the top for easy typing.

**🌐 Web Search Integration**

- Uses DuckDuckGo Search to fetch real-time web results.


###**Displays:**
- DuckDuckgo Search Results

  - Title of the page

  - Short description/body

  - Direct link to the source

- 🧠 LLM-based Summarization(Uses OpenAI's GPT-4o-mini to generate)

  - 📈 Market overview

  - 🔍 Trends and public sentiment

  - ✅ Optional SWOT analysis (if enough data found)

##**Install Dependencies**

In [1]:
!pip install openai gradio duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 43.4 MB/s eta 0:00:00


## **Retrieve OpenAI API Key from Secrets and set as an ENV**

In [2]:
# Retrieve the API key from Colab's secrets
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

# Set as an ENVIROMENT var
import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

## **Define a Model**

In [3]:
from openai import OpenAI
client = OpenAI()

##**Function to fetch Results from DuckDuckGo-Search based on Query**

In [4]:
from duckduckgo_search import DDGS

def fetch_duckduckgo_results(query, max_results=5):
    results_list = []
    full_text = ""
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=max_results)
        for r in results:
            title = r.get("title", "")
            body = r.get("body", "")
            link = r.get("href", "")
            result_text = f"🔗 {title}\n{body}\n{link}"
            results_list.append(result_text)
            full_text += f"{title}\n{body}\n"
    return results_list, full_text.strip()


##**Function for Summary Generation using LLM**

In [5]:
def summarize_with_llm(raw_text, topic):
    prompt = f"""
You are a market research assistant. Summarize the following search results about "{topic}" into:
- A brief market overview
- Recent trends and public interest
- SWOT analysis (optional if not enough data)

Text to analyze:
{raw_text}
"""
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # or gpt-3.5-turbo
        messages=[{"role": "user", "content": prompt}],
        max_tokens=500
    )
    return response.choices[0].message.content.strip()

##**Function for Analysis**

In [6]:
def run_analysis(topic):
    results_list, raw_text = fetch_duckduckgo_results(topic)
    summary = summarize_with_llm(raw_text, topic)
    return "\n\n".join(results_list), summary

def clear_fields():
    return "", ""

##**Gradio App**
###**Sample Input:**
- Tesla
- CloudYuga
- IT
- Patanjali Aloe-Vera gel etc.

In [7]:
import gradio as gr

# Gradio UI
with gr.Blocks(title="Market Research Assistant") as demo:
    gr.Markdown("## 📊 GenAI Market Research Assistant")
    gr.Markdown("Enter a product, brand, or industry to get web search results and a summarized report.")

    input_topic = gr.Textbox(label="🔍 Enter Topic", placeholder="e.g. electric scooters", lines=1)

    with gr.Row():
        analyze_btn = gr.Button("Analyze", variant="primary")
        clear_btn = gr.Button("Clear")

    with gr.Row():
        with gr.Column():
            result_box = gr.Textbox(label="📰 DuckDuckGo Results", lines=20, interactive=False)
        with gr.Column():
            summary_box = gr.Textbox(label="🧠 LLM Summary", lines=20, interactive=False)

    analyze_btn.click(run_analysis, inputs=input_topic, outputs=[result_box, summary_box])
    clear_btn.click(clear_fields, inputs=None, outputs=[result_box, summary_box])

if __name__ == "__main__":
    demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c217bad2b0a05ade41.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
